In [4]:
import tensorflow as tf
import numpy as np
import time

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="./ssd7_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

#Inference
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

#0=cls, 1=reg, 2=base_layers
tflite_results = interpreter.get_tensor(output_details[0]['index'])


In [5]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd7 import build_model
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
from data_generator.data_augmentation_chain_variable_input_size import DataAugmentationVariableInputSize
from data_generator.data_augmentation_chain_constant_input_size import DataAugmentationConstantInputSize
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation

from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

Using TensorFlow backend.


In [6]:
# TODO: Set the path to the `.h5` file of the model to be loaded.
model_path = './ssd7_voc_person/ssd7_epoch-122_loss-2.0452_val_loss-1.9801.h5'
#model_path = "./ssd7_trained/ssd7_epoch-02_loss-3.0883_val_loss-2.9671.h5"

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

K.clear_session() # Clear previous models from memory.

tf_model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'compute_loss': ssd_loss.compute_loss})
















Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.




In [9]:
tf_results = model.predict(input_data)

In [16]:
for tf_result, tflite_result in zip(tf_results, tflite_results):
    decimal = 2 # @3 decimal places they different
    error = np.testing.assert_almost_equal(tf_result, tflite_result, decimal=decimal)
    
    #IF no assertion error raised...
    if error==None:
        print("TF and TFLITE models are equal up to {} decimal places".format(decimal))

TF and TFLITE models are equal up to 2 decimal places
